In [6]:
from surprise import Reader, Dataset, KNNWithMeans, accuracy
from surprise.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
import pandas as pd
import numpy as np

In [4]:
# 데이터 로드 및 전처리
df = pd.read_csv('../review_rating_hellody.csv', encoding = 'cp949')
df.reset_index(inplace=True, drop=True)
df = df[['user_id', 'movie_id', 'rating']]

train, test = train_test_split(df, test_size=0.2, random_state=42)
# 평점 범위를 1~10으로 설정
# 학습 및 테스트 데이터셋 분리
reader = Reader(rating_scale=(1, 10))
traindata = Dataset.load_from_df(train, reader)
testdata = Dataset.load_from_df(test, reader)

In [23]:
# 특정 사용자에 대한 영화 추천 함수
def get_recommendations(algo, user_id, df, n=20):
    # 사용자가 평가하지 않은 영화 목록 생성
    user_rated_movies = df[df['user_id'] == user_id]['movie_id'].tolist()
    all_movies = df['movie_id'].unique()
    unrated_movies = [movie for movie in all_movies if movie not in user_rated_movies]
    
    # 예상 평점 계산
    predictions = [algo.predict(user_id, movie) for movie in unrated_movies]
    predictions.sort(key=lambda x: x.est, reverse=True) #예상 평점 기준 정렬
    
    # 상위 n개의 영화 추천
    top_n_recommendations = predictions[:n]
    return [(pred.iid, pred.est) for pred in top_n_recommendations] # (영화id, 예상 평점)

BaselineOnly

###### 사용자의 평균 평점과 사용자/항목의 편향을 반영

In [7]:
from surprise import BaselineOnly

# 하이퍼파라미터 그리드 설정
param_grid = {
    'bsl_options': {
        'method': ['als', 'sgd'],
        'learning_rate' : [0.01, 0.001]
        'n_epochs': [30, 50, 70],
        'reg_u': [10, 15],
        'reg_i': [5, 10]
    }
}

# GridSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
gs = GridSearchCV(BaselineOnly, param_grid, measures=['rmse'], cv=5, n_jobs=-1) # 5-폴드 교차 검증
gs.fit(traindata)

# 최적의 하이퍼파라미터와 해당 성능 출력
print("Best RMSE score: ", gs.best_score['rmse'])
print("Best parameters: ", gs.best_params['rmse'])

Best RMSE score:  2.509871877398696
Best parameters:  {'bsl_options': {'method': 'sgd', 'n_epochs': 30, 'reg_u': 10, 'reg_i': 5}}


In [8]:
bsl_options = {'method' : 'sgd',
               'n_epochs' : 30,
               'reg_u' : 10,
               'reg_i' : 5}

algo = BaselineOnly(bsl_options = bsl_options)
predictions = algo.fit(traindata).test(testdata)
accuracy.rmse(predictions)

Estimating biases using sgd...


AttributeError: 'DatasetAutoFolds' object has no attribute 'n_users'

In [ ]:
trainset = algo.traindata

In [ ]:
df = pd.DataFrame(predictions, columns = ['uid', 'iid', 'rui', 'est', 'details'])
df['err'] = abs(df.est - df.rui)

best_predictions = df.sort_values(by = 'err')[:10]
worst_predictions = df.sort_values(by = 'err')[-10:]

In [26]:
from surprise import Dataset, Reader, KNNWithMeans

# 하이퍼파라미터 그리드 설정
param_grid = {
    'k': [5, 10, 20, 30, 40, 50],  # 이웃 수
    'sim_options': {
        'name': ['pearson_baseline'],
        'user_based': [True]
    }
}

# GridSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
gs = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], cv=5, n_jobs=-1) # 5-폴드 교차 검증
gs.fit(traindata)

# 최적의 하이퍼파라미터와 해당 성능 출력
print("Best RMSE score: ", gs.best_score['rmse'])
print("Best parameters: ", gs.best_params['rmse'])

Best RMSE score:  2.3767002478046115
Best parameters:  {'k': 10, 'sim_options': {'name': 'pearson_baseline', 'user_based': True}}


In [25]:
# KNN 모델 학습
algo = KNNWithMeans(k=30, sim_options={'name': 'pearson_baseline', 'user_based': True})
algo.fit(trainset)

# 테스트 데이터셋으로 예측 및 RMSE 계산
test_pred = algo.test(testset)
print("Item-based Model : Test Set")
accuracy.rmse(test_pred, verbose=True)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Item-based Model : Test Set
RMSE: 2.8523


2.8523430209193466

In [7]:
# 사용자 ID에 대한 추천 영화 목록 출력
user_id = 2  
recommended_movies = get_recommendations(algo, user_id, df, n=20)
print("추천된 영화 목록:")
for movie_id, est_rating in recommended_movies:
    print(f"영화 ID: {movie_id}, 예상 평점: {est_rating}")

추천된 영화 목록:
영화 ID: 100015, 예상 평점: 10
영화 ID: 10002, 예상 평점: 10
영화 ID: 100021, 예상 평점: 10
영화 ID: 100022, 예상 평점: 10
영화 ID: 100023, 예상 평점: 10
영화 ID: 100026, 예상 평점: 10
영화 ID: 10003, 예상 평점: 10
영화 ID: 100039, 예상 평점: 10
영화 ID: 10004, 예상 평점: 10
영화 ID: 100041, 예상 평점: 10
영화 ID: 10005, 예상 평점: 10
영화 ID: 10006, 예상 평점: 10
영화 ID: 100069, 예상 평점: 10
영화 ID: 10007, 예상 평점: 10
영화 ID: 100075, 예상 평점: 10
영화 ID: 10008, 예상 평점: 10
영화 ID: 100084, 예상 평점: 10
영화 ID: 100086, 예상 평점: 10
영화 ID: 100087, 예상 평점: 10
영화 ID: 10009, 예상 평점: 10


KNNBaseline

###### knnbaseline - gridsearch

In [9]:
from surprise import Dataset, Reader, KNNBaseline

# 하이퍼파라미터 그리드 설정
param_grid = {
    'k': [5, 10, 20, 30, 40, 50],  # 이웃 수
    'sim_options': {
        'name': ['pearson_baseline'],
        'user_based': [True]
    },
    'bsl_options': {
        'method': ['als', 'sgd'],
        'n_epochs': [30, 40, 50, 60],
        'reg_u': [10, 15],
        'reg_i': [5, 10]
    }
}

# GridSearchCV를 사용하여 최적의 하이퍼파라미터 찾기
gs = GridSearchCV(KNNBaseline, param_grid, measures=['rmse'], cv=5, n_jobs=-1) # 5-폴드 교차 검증
gs.fit(traindata)

# 최적의 하이퍼파라미터와 해당 성능 출력
print("Best RMSE score: ", gs.best_score['rmse'])
print("Best parameters: ", gs.best_params['rmse'])

Best RMSE score:  2.3177131267240956
Best parameters:  {'k': 30, 'sim_options': {'name': 'pearson_baseline', 'user_based': True}, 'bsl_options': {'method': 'sgd', 'n_epochs': 40, 'reg_u': 10, 'reg_i': 5}}


In [10]:
sim_options = {'name': 'pearson_baseline', 'user_based': True}
bsl_options = {'method': 'sgd', 'n_epochs': 40, 'reg_u': 10, 'reg_i': 5}
algo = KNNBaseline(k=30, sim_options=sim_options, bsl_options=bsl_options)
algo.fit(trainset)

# 테스트 데이터셋으로 예측 및 RMSE 계산
test_pred = algo.test(testset)
print("Item-based Model : Test Set")
accuracy.rmse(test_pred, verbose=True)

Estimating biases using sgd...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Item-based Model : Test Set
RMSE: 2.2759


2.2758626961102832

In [11]:
# 사용자 ID에 대한 추천 영화 목록 출력
user_id = 2  
recommended_movies = get_recommendations(algo, user_id, df, n=20)
print("추천된 영화 목록:")
for movie_id, est_rating in recommended_movies:
    print(f"영화 ID: {movie_id}, 예상 평점: {est_rating}")

추천된 영화 목록:
영화 ID: 10016, 예상 평점: 10
영화 ID: 10038, 예상 평점: 10
영화 ID: 100399, 예상 평점: 10
영화 ID: 10058, 예상 평점: 10
영화 ID: 100610, 예상 평점: 10
영화 ID: 10072, 예상 평점: 10
영화 ID: 100825, 예상 평점: 10
영화 ID: 10002, 예상 평점: 9.965172679776229
영화 ID: 100398, 예상 평점: 9.889670124706875
영화 ID: 10048, 예상 평점: 9.88064651722792
영화 ID: 10028, 예상 평점: 9.827168473624713
영화 ID: 10035, 예상 평점: 9.808704918333367
영화 ID: 10006, 예상 평점: 9.760100933553552
영화 ID: 10020, 예상 평점: 9.751880166464144
영화 ID: 10018, 예상 평점: 9.748104958748288
영화 ID: 100619, 예상 평점: 9.746221374804085
영화 ID: 10047, 예상 평점: 9.71933728626436
영화 ID: 10046, 예상 평점: 9.719129491212529
영화 ID: 10021, 예상 평점: 9.71906072757728
영화 ID: 10005, 예상 평점: 9.705452394657641
